In [479]:
import numpy as np
import pandas as pd
import talib
import numpy
import datetime as dt
pd.set_option('display.max_rows', None)  # show all rows
pd.set_option('display.max_columns', None)  # show all columns
pd.set_option('display.width', None)  # don't wrap lines
pd.set_option('display.max_colwidth', None)  # show full text in cells

df = pd.read_excel(r'gbp_usd_1h.xlsx')
df['atr'] = talib.ATR(
    df['high'],
    df['low'],
    df['close'],
    timeperiod=24
)
df['ma'] = talib.KAMA(
    df['close'],
    timeperiod=14
)

df['adx'] = talib.ADX(
    df['high'],
    df['low'],
    df['close'],
    timeperiod=24
)

df['rsi'] = talib.RSI(
    df['close'],
    timeperiod=14
)

df['macd'] = talib.RSI(
    df['close'],
    timeperiod=14
)


In [480]:
df = df.iloc[47:].reset_index(drop=True)

In [481]:
df = df.drop(columns=['Unnamed: 0'])

In [482]:
def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'


df['ma_status'] = df.apply(set_ma_status, axis=1)

In [483]:
df['id'] = df.index

In [484]:
import numpy as np

In [485]:
df['entry'] = df['close']
df['tp_buy'] = df['close'] + (df['atr'] * 2.4)
df['tp_sell'] = df['close'] - (df['atr'] * 2.4)

In [486]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['timestamp'] = df['datetime'].apply(lambda d: int(d.timestamp()))

In [487]:
array = np.array(df[['timestamp', 'open', 'high', 'low', 'close', 'atr', 'adx', 'tp_buy', 'tp_sell']])

In [488]:
patterns = []
array_size = len(array)

for i, element in enumerate(array):
    if i + 7 == array_size:
        break

    pat = array[i:i + 7, :]
    patterns.append(pat)

In [489]:
def get_trade_from_pattern(pattern: np.array):
    timestamp = pattern[-1, 0].copy()
    entry = pattern[-1, 4].copy()
    tp_buy = pattern[-1, 7].copy()
    tp_sell = pattern[-1, 8].copy()

    return timestamp, entry, tp_buy, tp_sell


any_trade = get_trade_from_pattern(patterns[10])

In [490]:
def evaluate_trade_result(chart: np.array, trade: tuple):
    timestamp, entry, tp_buy, tp_sell = trade

    backtest_chart = chart[chart[:, 0] >= timestamp]

    for i, element in enumerate(backtest_chart):
        price_high = element[2]
        price_low = element[3]

        # print(f'entry: {entry} price_high: {price_high} price_low: {price_low}')
        # print(f'tp_buy: {tp_buy} tp_sell: {tp_sell}')

        if price_high > tp_buy:
            return "buy"

        if price_low < tp_sell:
            return "sell"

    return "None"

In [491]:
trade_result = []
for pat in patterns:
    trade = get_trade_from_pattern(pat)
    result = evaluate_trade_result(array, trade)
    trade_result.append((trade[0], result))

In [492]:
patterns_for_train = []
for pat in patterns:
    id = pat[-1, 0]
    closes = pat[:, 4]
    atr = pat[-1, 5]
    adx = pat[-1, 6]
    result = np.array([id, atr, adx])
    result = np.hstack((result, closes))
    patterns_for_train.append(result)

In [493]:
patterns_for_train_df = pd.DataFrame(patterns_for_train, columns=[
    'timestamp',
    'atr',
    'adx',
    'close_1',
    'close_2',
    'close_3',
    'close_4',
    'close_5',
    'close_6',
    'close_7',
])

In [494]:
trade_result_df = pd.DataFrame(trade_result, columns=['timestamp', 'result'])

In [495]:
patterns_for_train_df = pd.merge(patterns_for_train_df, trade_result_df, on='timestamp')

In [496]:
df_ma_statuses = df[['timestamp', 'ma_status', 'macd', 'rsi']]

In [497]:
final_patterns_for_training = pd.merge(patterns_for_train_df, df_ma_statuses, on='timestamp')

In [498]:
final_patterns_for_training = final_patterns_for_training[
    final_patterns_for_training['result'] != 'None'
    ]

In [499]:
final_patterns_for_training['result'] = (
    final_patterns_for_training['result'].apply(lambda x: 1 if x == 'buy' else 0)
)

In [500]:
final_patterns_for_training['ma_status'] = (
    final_patterns_for_training['ma_status'].apply(lambda x: 1 if x == 'upper' else 0)
)

In [501]:
final_patterns_for_training.columns

Index(['timestamp', 'atr', 'adx', 'close_1', 'close_2', 'close_3', 'close_4',
       'close_5', 'close_6', 'close_7', 'result', 'ma_status', 'macd', 'rsi'],
      dtype='object')

In [502]:
data_set = final_patterns_for_training[
    ['timestamp', 'atr', 'adx', 'ma_status', 'macd', 'rsi', 'close_1', 'close_2', 'close_3', 'close_4', 'close_5',
     'close_6', 'close_7', 'result']].copy()

In [503]:
data_set['previous_result_1'] = data_set['result'].shift(1)
data_set['previous_result_2'] = data_set['result'].shift(2)
data_set['previous_result_3'] = data_set['result'].shift(3)


data_set['previous_result_sum'] = \
    (
            (data_set['previous_result_1'] * 3) +
            (data_set['previous_result_2'] * 2) +
            data_set['previous_result_3']
    )


In [504]:
data_set = data_set.iloc[5:]

In [505]:
data_set = data_set.reset_index(drop=True)

In [506]:
data_set.head(5)

,timestamp,atr,adx,ma_status,macd,rsi,close_1,close_2,close_3,close_4,close_5,close_6,close_7,result,previous_result_1,previous_result_2,previous_result_3,previous_result_sum
0,1.578301e+09,0.001878,33.496691,0,39.818787,39.818787,1.30782,1.30866,1.30811,1.30830,1.30804,1.30818,1.30719,1,1.0,1.0,1.0,6.0
1,1.578305e+09,0.001985,32.393252,1,54.709231,54.709231,1.30866,1.30811,1.30830,1.30804,1.30818,1.30719,1.31046,1,1.0,1.0,1.0,6.0
2,1.578308e+09,0.002092,31.620912,1,65.309660,65.309660,1.30811,1.30830,1.30804,1.30818,1.30719,1.31046,1.31421,1,1.0,1.0,1.0,6.0
3,1.578312e+09,0.002153,31.278805,1,67.350579,67.350579,1.30830,1.30804,1.30818,1.30719,1.31046,1.31421,1.31514,1,1.0,1.0,1.0,6.0
4,1.578316e+09,0.002161,30.965834,1,68.279808,68.279808,1.30804,1.30818,1.30719,1.31046,1.31421,1.31514,1.31557,1,1.0,1.0,1.0,6.0
